In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os
import json
import random
# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)


import tensorflow as tf
#from tensorflow import keras as tfk
import keras as tfk       #notice how I'm importing keras and not tensorflow.keras
from keras.layers import Input, Dense, Dropout, Lambda
#from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from keras import layers as tfkl
import keras_cv


print(f"Tensorflow version -> {tf.__version__}")
print(f"Keras version -> {tfk.__version__}")
# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

Tensorflow version -> 2.16.1
Keras version -> 3.3.3
2.16.1


In [2]:
X_train = np.load("/kaggle/input/datasetmax/processed_dataMax/X_train.npy")
y_train = np.load("/kaggle/input/datasetmax/processed_dataMax/y_train.npy")
X_val = np.load("/kaggle/input/datasetmax/processed_dataMax/X_val.npy")
y_val = np.load("/kaggle/input/datasetmax/processed_dataMax/y_val.npy")

indices = np.arange(X_train.shape[0])  # Create an array of indices
np.random.shuffle(indices)  # Shuffle the indices

# Apply the shuffled indices to both X_train and y_train
X_train = X_train[indices]
y_train = y_train[indices]

print("Train shapes: ",X_train.shape," ",y_train.shape)
print("Validation shapes: ",X_val.shape," ",y_val.shape)

Train shapes:  (39687, 96, 96, 3)   (39687,)
Validation shapes:  (1196, 96, 96, 3)   (1196, 1)


In [3]:
def save_model(code, model, history, folder_name):

    """

    Salva il modello e i parametri in una cartella specificata.



    Args:

    - model: il modello da salvare

    - params: dizionario contenente i parametri da salvare (es. learning rate, batch size, etc.)

    - folder_name: nome della cartella di destinazione (default: 'model_folder')

    """

    # Crea la cartella se non esiste

    os.makedirs(folder_name, exist_ok=True)



    if code == 0:

          model_save_path = os.path.join(folder_name, 'weightsTL.keras')

          model.save(model_save_path)

          print(f"ModelTL saved at: {model_save_path}")

    else:

         model_save_path = os.path.join(folder_name, 'weights.keras')

         model.save(model_save_path)

         print(f"Model saved at: {model_save_path}")



    # Salvataggio della history in un file JSON

    if code == 0:

       history_save_path = os.path.join(folder_name, 'historyTL.json')

       with open(history_save_path, 'w') as f:

           json.dump(history, f, indent=4)

       print(f"HistoryTL saved at: {history_save_path}")

    else:

      history_save_path = os.path.join(folder_name, 'history.json')

      with open(history_save_path, 'w') as f:

           json.dump(history, f, indent=4)

      print(f"History saved at: {history_save_path}")

In [4]:
# Define a mapping of labels to their corresponding cell type names
labels = {
    0: 'Basophil',
    1: 'Eosinophil',
    2: 'Erythroblast',
    3: 'Immature granulocytes',
    4: 'Lymphocyte',
    5: 'Monocyte',
    6: 'Neutrophil',
    7: 'Platelet'
}
# Save unique labels
unique_labels = list(labels.values())

In [5]:
from sklearn.utils import class_weight

# Calcola i pesi delle classi
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train.flatten()), y=y_train.flatten())
class_weight_dict = dict(enumerate(class_weights))

print("Class weights:", class_weight_dict)

Class weights: {0: 1.1264475476839237, 1: 0.8432559918408975, 2: 1.1264475476839237, 3: 0.9085851648351648, 4: 1.1264475476839237, 5: 1.1264475476839237, 6: 0.7885670004768717, 7: 1.1180696416497633}


In [6]:
# Convert class labels to categorical format for training, validation, and test sets
y_train = tfk.utils.to_categorical(y_train, num_classes=len(unique_labels))
y_val = tfk.utils.to_categorical(y_val, num_classes=len(unique_labels))



# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (39687, 96, 96, 3), y_train shape: (39687, 8)
X_val shape: (1196, 96, 96, 3), y_val shape: (1196, 8)


In [7]:
# Input shape for the model
input_shape = X_train.shape[1:]

# Output shape for the model
output_shape = y_train.shape[-1]

print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

Input Shape: (96, 96, 3)
Output Shape: 8


In [8]:
# Batch size for training
batch_size = 128

# Learning rate: step size for updating the model's weights
learning_rate = 1e-4


l2_lambda = 1e-4

# Augmentation: set an augmentation layer or not
augmentation = True

# Patience
patience = 10

folder_name = "ConvNeXtSmall_V3"

# Dropout

#Name
name = 'ConvNeXtSmall_V3'

#Display the architecture
display = False

# Print the defined parameters
print("Batch Size:", batch_size)
print("Learning Rate:", learning_rate)
print("Augmentation:", augmentation)
print("Patience:", patience)

Batch Size: 128
Learning Rate: 0.0001
Augmentation: True
Patience: 10


In [9]:

# Base model with stochastic depth
initializer = tfk.initializers.GlorotNormal(seed=seed)
regulariser = tfk.regularizers.l2(l2_lambda)

tl_model = tfk.applications.ConvNeXtSmall(
    include_top=False,
    include_preprocessing=True,  # Handles preprocessing internally
    weights="imagenet",
    input_shape=input_shape,
    pooling="avg",
)
tl_model.trainable = False

# Input preprocessing
inputs = tfkl.Input(shape=(None, None, 3))  # Handle arbitrary input sizes



x = tfkl.Resizing(96, 96)(inputs)


# Feature extraction
x = tl_model(x)

# Stronger regularization in head
x = tfkl.Dense(512,kernel_initializer= initializer)(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(0.5)(x)

x = tfkl.Dense(256,kernel_initializer= initializer)(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(0.5)(x)

# Output with label smoothing
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer= initializer,
    kernel_regularizer=regulariser
)(x)

tl_model = tfk.Model(inputs=inputs, outputs=outputs, name=name)

# Cosine decay with warmup
# Cosine decay schedule
lr_schedule = tfk.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=learning_rate,
    first_decay_steps=500,
    t_mul=1.5,
    m_mul=0.85,
    alpha=0.05
)


tl_model.compile(
    optimizer=tfk.optimizers.AdamW(learning_rate=lr_schedule,weight_decay=l2_lambda ),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2),
    metrics=['accuracy']
)


198551472/198551472 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [10]:
tl_history = tl_model.fit(
        X_train,
        y_train,
        batch_size=128,
        validation_data=(X_val, y_val),
        epochs=3,
        class_weight=class_weight_dict,
    ).history


# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
save_model(0,tl_model, tl_history, folder_name)  #+ str(final_val_accuracy)

# Free memory by deleting the model instance

del tl_model

Epoch 1/3


I0000 00:00:1732029873.986634      70 service.cc:145] XLA service 0x7d4c5c097a30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732029873.987392      70 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  1/311 ━━━━━━━━━━━━━━━━━━━━ 2:07:21 25s/step - accuracy: 0.1328 - loss: 2.7662

I0000 00:00:1732029883.664887      70 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


311/311 ━━━━━━━━━━━━━━━━━━━━ 70s 146ms/step - accuracy: 0.2247 - loss: 2.2748 - val_accuracy: 0.6329 - val_loss: 1.6305
Epoch 2/3
311/311 ━━━━━━━━━━━━━━━━━━━━ 31s 100ms/step - accuracy: 0.3997 - loss: 1.8223 - val_accuracy: 0.6948 - val_loss: 1.4924
Epoch 3/3
311/311 ━━━━━━━━━━━━━━━━━━━━ 31s 100ms/step - accuracy: 0.4834 - loss: 1.7027 - val_accuracy: 0.7567 - val_loss: 1.3617
Final validation accuracy: 75.67%
ModelTL saved at: ConvNeXtSmall_V3/weightsTL.keras
HistoryTL saved at: ConvNeXtSmall_V3/historyTL.json


In [11]:
ft_model = tfk.models.load_model(folder_name+'/weightsTL.keras')
# Display a summary of the model architecture
#ft_model.summary(expand_nested=True,show_trainable=True)

# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [12]:
# Unfreeze only the last 20 layers of the ConvNeXt backbone
convnext_layers = [
    layer for layer in ft_model.get_layer("convnext_small").layers
]

print(len(convnext_layers))

# Freeze all layers initially
for layer in convnext_layers:
    layer.trainable = False

# Unfreeze the last 40 layers
for layer in convnext_layers[-40:]:
    layer.trainable = True
    #print(layer.name, type(layer).__name__, layer.trainable)

#ft_model.summary(expand_nested=True,show_trainable=True)

260


In [13]:
# Cosine decay with warmup
# Cosine decay schedule
lr_schedule = tfk.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=learning_rate,
    first_decay_steps=500,
    t_mul=1.5,
    m_mul=0.85,
    alpha=0.05
)


ft_model.compile(
    optimizer=tfk.optimizers.AdamW(learning_rate=lr_schedule,weight_decay=l2_lambda ),
    loss=tfk.losses.CategoricalCrossentropy(label_smoothing=0.2),
    metrics=['accuracy']
)

In [14]:
ft_history = ft_model.fit(
        X_train,
        y_train,
        batch_size=128,
        validation_data=(X_val, y_val),
        epochs=3000,
        class_weight=class_weight_dict,
        
        callbacks = [
            tfk.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True
            )
        ]
        
    ).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(ft_history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
save_model(1,ft_model, ft_history, folder_name)

# Delete the model to free up resources
del ft_model

Epoch 1/3000
  1/311 ━━━━━━━━━━━━━━━━━━━━ 3:13:25 37s/step - accuracy: 0.4766 - loss: 1.6326

I0000 00:00:1732030082.956063      71 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_transpose_fusion_140', 4 bytes spill stores, 4 bytes spill loads



311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6428 - loss: 1.4719

I0000 00:00:1732030133.295129      70 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_transpose_fusion_140', 4 bytes spill stores, 4 bytes spill loads



311/311 ━━━━━━━━━━━━━━━━━━━━ 95s 185ms/step - accuracy: 0.6430 - loss: 1.4716 - val_accuracy: 0.9181 - val_loss: 1.0184
Epoch 2/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.7822 - loss: 1.2649 - val_accuracy: 0.9457 - val_loss: 0.9740
Epoch 3/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.8255 - loss: 1.1971 - val_accuracy: 0.9599 - val_loss: 0.9384
Epoch 4/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.8607 - loss: 1.1380 - val_accuracy: 0.9624 - val_loss: 0.9273
Epoch 5/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.8709 - loss: 1.1206 - val_accuracy: 0.9682 - val_loss: 0.9095
Epoch 6/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.8945 - loss: 1.0770 - val_accuracy: 0.9666 - val_loss: 0.8972
Epoch 7/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.9108 - loss: 1.0456 - val_accuracy: 0.9716 - val_loss: 0.8889
Epoch 8/3000
311/311 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.9186 - loss

In [ ]:
from sklearn.metrics import classification_report
model = tfk.models.load_model(folder_name+'/weights.keras')
predictions = model.predict(X_val)
predicted_classes = np.argmax(predictions, axis=1)
print(classification_report(np.argmax(y_val,axis=-1), predicted_classes))
